In [24]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install gym[atari]


    100% |████████████████████████████████| 28.2MB 339kB/s ta 0:00:011
    100% |████████████████████████████████| 2.8MB 2.5MB/s ta 0:00:011


## Importing the packages

In [4]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

## Creating the Environment

In [27]:
env = gym.make("FrozenLake-v0")

## Creating the Q-Table

In [28]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

In [29]:
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Initialising Q-Learning Parameters

In [30]:
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

## Coding the Q-learning algorithm training loop

In [31]:
rewards_all_episodes = []

In [32]:
# Q-learning algorithm
for episode in range(num_episodes):
    # initialize new episode params
    state = env.reset()
    done = False
    rewards_current_episode = 0
    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        else:
            action = env.action_space.sample()
        # Take new action
        new_state, reward, done, info = env.step(action)
        # Update Q-table
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
        learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        # Set new state
        state = new_state
        # Add new reward        
        rewards_current_episode += reward
        if done == True: 
            break
    # Exploration rate decay
    exploration_rate = min_exploration_rate + \
    (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    # Add current episode reward to total rewards list
    rewards_all_episodes.append(rewards_current_episode)

## Calculate and print the average reward per thousand episodes

In [33]:
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  0.035000000000000024
2000 :  0.20600000000000016
3000 :  0.3920000000000003
4000 :  0.5480000000000004
5000 :  0.6390000000000005
6000 :  0.6440000000000005
7000 :  0.6590000000000005
8000 :  0.6500000000000005
9000 :  0.7130000000000005
10000 :  0.6590000000000005


## Print Updated Q-Table

In [34]:
print("\n\n********Q-table********\n")
print(q_table)




********Q-table********

[[0.54242714 0.4799767  0.48708125 0.48903472]
 [0.36468006 0.36235112 0.27154549 0.46260728]
 [0.39575037 0.30353112 0.24841091 0.27568645]
 [0.06148557 0.07807978 0.0238682  0.11456737]
 [0.55828724 0.25989324 0.32166768 0.30581934]
 [0.         0.         0.         0.        ]
 [0.2064396  0.13344864 0.25370363 0.03651276]
 [0.         0.         0.         0.        ]
 [0.41619755 0.2975589  0.40587613 0.58092794]
 [0.36125134 0.60607461 0.44192996 0.45048281]
 [0.48954404 0.42171053 0.27415129 0.21252455]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.37645113 0.51120837 0.75588565 0.53427258]
 [0.68985548 0.85655946 0.75618331 0.7084555 ]
 [0.         0.         0.         0.        ]]


## Watching the agent play the game

In [35]:
# Watch our agent play Frozen Lake by playing the best action 
# from each state according to the Q-table

for episode in range(3):
    # initialize new episode params
    state = env.reset()
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)
    for step in range(max_steps_per_episode):        
        # Show current state of environment on screen
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        # Choose action with highest Q-value for current state
        action = np.argmax(q_table[state,:])
        # Take new action
        new_state, reward, done, info = env.step(action)

        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                # Agent reached the goal and won episode
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                # Agent stepped in a hole and lost episode
                print("****You fell through a hole!****")
                time.sleep(3)
                clear_output(wait=True)
            break
        # Set new state
        state = new_state
env.close()

  (Right)
SFFF
FHFH
FFFH
HFFG
****You fell through a hole!****
